<a href="https://colab.research.google.com/github/billysco/sentiment_analysis/blob/Aruna/Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import numpy as np
import os
import requests
import json
from sklearn import svm
from sklearn.preprocessing import StandardScaler


In [3]:
!pip install alpaca-trade-api


     |████████████████████████████████| 81kB 3.7MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 


In [9]:
!pip install python-dotenv

In [10]:
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [12]:
# Load .env environment variables
load_dotenv()

# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Check to make sure keys imported correctly, str confirms success
display(type(alpaca_api_key))
display(type(alpaca_secret_key))

str

str

In [13]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [15]:
# Set date range as ISO format, the end date needs to be updated as we go
start_date = pd.Timestamp("2020-11-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2021-04-14", tz="America/New_York").isoformat()

In [16]:
# Create tickers variable, these should be updated as needed
tickers = ['GME', 'AMC', 'TSLA', 'TLRY', 'SNDL', 'PLTR']

In [17]:
# Set timeframe to one day ('1D') to get daily data
timeframe = "1D"

In [19]:
# Get current closing prices for the tickers
df_stonks = alpaca.get_barset(
    tickers,
    timeframe,
    start = start_date,
    end = end_date
).df

# Display sample data
df_stonks.head()

AMC               ...      TSLA                  
                           open   high   low  ...       low   close    volume
time                                          ...                            
2020-11-18 00:00:00-05:00  3.08  3.370  3.01  ...  443.5001  486.64  75901661
2020-11-19 00:00:00-05:00  3.16  3.375  3.15  ...  487.5700  499.16  59759624
2020-11-20 00:00:00-05:00  3.25  3.370  3.21  ...  489.0600  489.44  31084668
2020-11-23 00:00:00-05:00  3.51  3.850  3.41  ...  501.7900  521.67  46506862
2020-11-24 00:00:00-05:00  4.16  5.000  4.15  ...  526.2000  555.86  49296561

[5 rows x 30 columns]

In [20]:
# Extract closing data for all stocks, start with an empty DataFrame
closing_prices_df = pd.DataFrame()

# Get the closing prices of stocks
closing_prices_df["GME"] = df_stonks["GME"]["close"]
closing_prices_df["AMC"] = df_stonks["AMC"]["close"]
closing_prices_df["TSLA"] = df_stonks["TSLA"]["close"]
closing_prices_df["TLRY"] = df_stonks["TLRY"]["close"]
closing_prices_df["SNDL"] = df_stonks["SNDL"]["close"]
closing_prices_df["PLTR"] = df_stonks["PLTR"]["close"]

# Drop the time component of the date
closing_prices_df.index = closing_prices_df.index.date

# Display sample data
display(closing_prices_df.head())
display(closing_prices_df.tail())

,GME,AMC,TSLA,TLRY,SNDL,PLTR
2020-11-18,11.58,3.26,486.64,7.140,0.2589,17.90
2020-11-19,12.46,3.19,499.16,6.730,0.2623,18.98
2020-11-20,12.71,3.35,489.44,7.350,0.2544,18.14
2020-11-23,13.90,3.80,521.67,6.675,0.2417,21.03
2020-11-24,13.68,4.58,555.86,7.810,0.3000,23.85


,GME,AMC,TSLA,TLRY,SNDL,PLTR
2021-04-08,170.24,9.7800,683.77,19.7600,1.015000,23.4100
2021-04-09,158.32,9.4200,677.02,19.7600,1.000000,24.0301
2021-04-12,141.07,8.6203,701.63,17.1500,0.931499,23.3300
2021-04-13,140.72,8.8411,762.35,18.3199,0.976400,25.4150
2021-04-14,167.00,9.3500,732.99,17.3950,0.905700,23.6800


In [21]:
# Extract open data for all stocks, start with an empty DataFrame
opening_prices_df = pd.DataFrame()

# Get the opening prices of stocks
opening_prices_df["GME"] = df_stonks["GME"]["open"]
opening_prices_df["AMC"] = df_stonks["AMC"]["open"]
opening_prices_df["TSLA"] = df_stonks["TSLA"]["open"]
opening_prices_df["TLRY"] = df_stonks["TLRY"]["open"]
opening_prices_df["SNDL"] = df_stonks["SNDL"]["open"]
opening_prices_df["PLTR"] = df_stonks["PLTR"]["open"]

# Drop the time component of the date
opening_prices_df.index = opening_prices_df.index.date

# Display sample data
display(closing_prices_df.head())
display(closing_prices_df.tail())

,GME,AMC,TSLA,TLRY,SNDL,PLTR
2020-11-18,11.58,3.26,486.64,7.140,0.2589,17.90
2020-11-19,12.46,3.19,499.16,6.730,0.2623,18.98
2020-11-20,12.71,3.35,489.44,7.350,0.2544,18.14
2020-11-23,13.90,3.80,521.67,6.675,0.2417,21.03
2020-11-24,13.68,4.58,555.86,7.810,0.3000,23.85


,GME,AMC,TSLA,TLRY,SNDL,PLTR
2021-04-08,170.24,9.7800,683.77,19.7600,1.015000,23.4100
2021-04-09,158.32,9.4200,677.02,19.7600,1.000000,24.0301
2021-04-12,141.07,8.6203,701.63,17.1500,0.931499,23.3300
2021-04-13,140.72,8.8411,762.35,18.3199,0.976400,25.4150
2021-04-14,167.00,9.3500,732.99,17.3950,0.905700,23.6800
